In [ ]:
# savePath = "/content/gdrive/MyDrive/Colab Notebooks/data_edu/16. LLM 파인튜닝/models/llama_tune_{now_str}"

In [ ]:
try:

    import google.colab
    inColab = True
except ImportError:
    inColab = False

In [ ]:
inColab

True

In [ ]:
if inColab == True:
    !pip install -U pandas==2.2.2 numpy==2.0.2 scipy==1.14.1 accelerate==1.6.0 peft==0.15.2 bitsandbytes==0.45.5 transformers==4.51.3 trl==0.16.1 datasets==3.5.0 tensorboard==2.19.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging
)
from transformers import AutoConfig,AutoModel
import torch
from peft import PeftModel, PeftConfig

In [ ]:
!pip show torch


Name: torch
Version: 2.6.0+cu124
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-cusparselt-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, bitsandbytes, fastai, peft, sentence-transformers, timm, torchaudio, torchvision


In [ ]:
if inColab == True:
    from google.colab import drive
    drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
import huggingface_hub
huggingface_hub.login()

# 1. Load base and new model

In [ ]:
# from transformers import AutoConfig, AutoModel, AutoTokenizer
# config = AutoConfig.from_pretrained("your model name", revision=revision)
# model = AutoModel.from_pretrained("your model name", revision=revision)
# tokenizer = AutoTokenizer.from_pretrained("your model name", revision=revision)

### ★★★ 수정 포인트 ★★★

In [ ]:
%cd /content/gdrive/MyDrive/Colab Notebooks/llm/llama_3.0_8B/checkpoint-3351

[Errno 2] No such file or directory: '/content/gdrive/MyDrive/Colab Notebooks/llm/llama_3.0_8B/checkpoint-3351'
/content/gdrive/MyDrive/Colab Notebooks/llm/llama_3.0_8B/checkpoint-3350


In [7]:
## base 모델
base_model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# base_model = model_id = "google/gemma-7b-it"
## 파인튜닝한 모델 저장 위치 설정
# new_model = "/content/gdrive/MyDrive/Colab Notebooks/fintech_edu_2025/04. LLM 파인튜닝/models/llama_tune_2025_04_18_05"
new_model = "C:/Users/SMT33/Desktop/dkdk_tinyllama" #어뎁터 위치한 폴더
# /content/gdrive/MyDrive/Colab Notebooks/llm/llama_3.0_8B/checkpoint-975

In [8]:
import torch
torch.cuda.empty_cache()  # GPU 캐시 비우기


In [9]:
### 베이스모델 불러오기
baseModel = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    # device_map= "auto"
    device_map= {"": 0}
)

### 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [10]:
# PEFT 설정 로드
peft_config = PeftConfig.from_pretrained(new_model)
print(peft_config)

# 베이스모델에 어댑터 적용
model = PeftModel.from_pretrained(baseModel, new_model)

LoraConfig(task_type='CAUSAL_LM', peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='TinyLlama/TinyLlama-1.1B-Chat-v1.0', revision=None, inference_mode=True, r=64, target_modules={'v_proj', 'q_proj'}, exclude_modules=None, lora_alpha=16, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', trainable_token_indices=None, loftq_config={}, eva_config=None, corda_config=None, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False), lora_bias=False)


### 어뎁터 연결 모델 정상작동 확인

In [11]:
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    model_kwargs={"torch_dtype": torch.float16},
    truncation=True
)
def extract_response_llama3(question):
    messages = [
        {"role": "system", "content": ""},
        {"role": "user", "content": question},
    ]

    prompt = pipe.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    terminators = [
        pipe.tokenizer.eos_token_id,
        pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipe(
        prompt,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.1,
        top_p=0.9,
        num_return_sequences=1
    )

    generated_text = outputs[0]['generated_text']
    response_lines = generated_text.strip().split('\n')
    meaningful_response = response_lines[-1]

    return meaningful_response
question = "체중 1kg 기준으로 수리야 나마스까 요가 운동 시 소비하는 에너지가 궁금"
response = extract_response_llama3(question)
print(response)

Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

3.0 kcal/kg


In [ ]:
#def generate_text_gemma(model, tokenizer, prompt, device="cuda:0"):
#    # 모델을 디바이스에 로드
#    # model.to(device)

#    # 프롬프트 인코딩
#   inputs = tokenizer(prompt, return_tensors="pt").to(device)

#    # 텍스트 생성
#    outputs = model.generate(
#        **inputs,
#        max_new_tokens=512,             # 생성할 최대 토큰 수
#        do_sample=True,                 # 샘플링 사용 여부
#        temperature=0.1,                # 샘플링 온도
#        eos_token_id=tokenizer.eos_token_id  # 종료 토큰 ID
#    )

#    # 생성된 텍스트 디코딩
#    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
#    return generated_text

## 예시: 프롬프트를 입력하여 텍스트 생성
#prompt = "<start_of_turn>체중 1kg 기준으로 수리야 나마스까 요가 운동 시 소비하는 에너지가 궁금해요.\n<end_of_turn>\n<start_of_turn>\n"
#result = generate_text_gemma(model, tokenizer, prompt)
#print(result)

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


<start_of_turn>유튜브 채널 hkcode는 누가 운영하나요?
<end_of_turn>
<start_of_turn>
채널 hkcode는 김효관 교수가 운영합니다.

<end_of_turn>


In [12]:
# 어댑터 병합
mergedModel = model.merge_and_unload()

In [13]:
# set your HF repository
hfAddr = "Ljk0501/TinyLlama_healthcare_exercise_model"

# save model and tokenizer to HF hub
mergedModel.push_to_hub(hfAddr)
tokenizer.push_to_hub(hfAddr)

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

C:\Users\SMT33\.pyenv\pyenv-win\versions\3.11.9\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SMT33\.cache\huggingface\hub\models--Ljk0501--TinyLlama_healthcare_exercise_model. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Ljk0501/TinyLlama_healthcare_exercise_model/commit/bc118797fc6f841a4457676336487b5dc9fdf409', commit_message='Upload tokenizer', commit_description='', oid='bc118797fc6f841a4457676336487b5dc9fdf409', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Ljk0501/TinyLlama_healthcare_exercise_model', endpoint='https://huggingface.co', repo_type='model', repo_id='Ljk0501/TinyLlama_healthcare_exercise_model'), pr_revision=None, pr_num=None)

★ 참고 코드

In [ ]:
def generate_text_gemma(model, tokenizer, prompt, device="cuda:0"):
    # 모델을 디바이스에 로드
    # model.to(device)

    # 프롬프트 인코딩
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # 텍스트 생성
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,             # 생성할 최대 토큰 수
        do_sample=True,                 # 샘플링 사용 여부
        temperature=0.1,                # 샘플링 온도
        eos_token_id=tokenizer.eos_token_id  # 종료 토큰 ID
    )

    # 생성된 텍스트 디코딩
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# 예시: 프롬프트를 입력하여 텍스트 생성
def extract_answer_from_output(output: str, query: str) -> str:
    # output 중에서 마지막 query 등장 이후 텍스트만 추출
    idx = output.rfind(query)
    if idx != -1:
        answer = output[idx + len(query):].strip()
    else:
        answer = output.strip()

    # 불필요한 개행 제거 후 첫 문장 또는 문단 추출
    answer = answer.lstrip('\n').strip()
    answer = answer.split('\n')[0].strip()
    return answer

# ✅ 실제 사용
query = "유튜브 채널 hkcode는 누가 운영하나요?"
prompt = f"<start_of_turn>{query}\n<end_of_turn>\n<start_of_turn>\n"
output = generate_text_gemma(mergedModel, tokenizer, prompt)
answer = extract_answer_from_output(output, query)
answer

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


'<end_of_turn>'

In [ ]:
### 리더보드 올리기전 테스트 코드 (모델 이상여부 확인, 에러 시 등록 X)

In [ ]:
from transformers import AutoConfig, AutoModel, AutoTokenizer
# hfAddr = "hyokwan/llama31_common"
config = AutoConfig.from_pretrained(hfAddr, revision="main")
model = AutoModel.from_pretrained(hfAddr, revision="main")
tokenizer = AutoTokenizer.from_pretrained(hfAddr, revision="main")

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]